In [1]:
!pip install langchain langchain-openai langgraph langgraph-checkpoint

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8

In [ ]:
# Checkpoint
# MemorySaver: 메모리에 체크포인트 정보를 저장 (영속화 X), langgraph-checkpoint 패키지를 설치해 사용
# SqliteSaver: SQLite 데이터베이스에 체크포인트 정보를 저장하는 체크포인터, langgraph-checkpoint-sqlite 패키지를 설치해 사용
# PostgresSaver: PostgreSQL 데이터베이스에 체크포인트 정보를 저장하는 체크포인터, 프로덕션에서 권장되며, langgraph-checkpoint-postgres 패키지를 설치해 사용

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [18]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 그래프의 스테이트 정의
class State(BaseModel):
  query: str
  messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

# 메시지를 추가하는 노드 함수
def add_message(state: State) -> dict[str, Any]:
  additional_messages = []
  if not state.messages:
    additional_messages.append(SystemMessage(content="당신은 최소한의 응답을 하는 대화 에이전트입니다."))
  additional_messages.append(HumanMessage(content=state.query))
  return {"messages": additional_messages}

# LLM 응답을 추가하는 노드 함수
def llm_response(state: State) -> dict[str, Any]:
  llm = ChatOpenAI(model="gpt-5-nano", temperature=0.5)
  ai_message = llm.invoke(state.messages)
  return {"messages": [ai_message]}

In [19]:
# 체크포인트 내용을 표시하는 함수 정의
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
  checkpoint_tuple = checkpointer.get_tuple(config)
  print("체크포인트 데이터:")
  pprint(checkpoint_tuple.checkpoint)
  print("\n메타데이터:")
  pprint(checkpoint_tuple.metadata)

In [20]:
# 그래프 정의 및 컴파일
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# 그래프 설정
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)
graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

checkpointer = MemorySaver()

compiled_graph = graph.compile(checkpointer=checkpointer)

In [21]:
# 실행해 작동 확인하기
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.")
first_response = compiled_graph.invoke(user_query, config)
first_response

{'query': '제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는 걸 기억하겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 44, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWaoL7Qw69ZOQZtfCOhF0uClExvtg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d001cda5-01ef-4ca1-9267-2f9b428f5918-0', usage_metadata={'input_tokens': 44, 'output_tokens': 411, 'total_tokens': 455, 'input_token_details': {'audio': 

In [22]:
for checkpoint in checkpointer.list(config):
  print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b6100-5463-68bc-8002-cf1b0f08be09'}}, checkpoint={'v': 4, 'ts': '2025-10-31T04:13:57.270523+00:00', 'id': '1f0b6100-5463-68bc-8002-cf1b0f08be09', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.9679181653339277', 'query': '00000000000000000000000000000002.0.9679181653339277', 'messages': '00000000000000000000000000000004.0.28057072467180166', 'branch:to:add_message': '00000000000000000000000000000003.0.03861687908867628', 'branch:to:llm_response': '00000000000000000000000000000004.0.28057072467180166'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.749836471415279'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000002.0.9679181653339277'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000003.0.03861687908867628'}}, 'updated_channels': ['messages'], 'channel_va

In [23]:
print_checkpoint_dump(checkpointer, config)

체크포인트 데이터:
{'channel_values': {'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는 걸 기억하겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 44, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWaoL7Qw69ZOQZtfCOhF0uClExvtg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d001cda5-01ef-4ca1-9267-2f9b428f5918-0', usage_metadata={'input_tokens': 44, 'output_tokens': 411, 'to

In [24]:
user_query = State(query="제가 좋아하는 것이 뭔지 기억하세요?")
second_response = compiled_graph.invoke(user_query, config)
second_response

{'query': '제가 좋아하는 것이 뭔지 기억하세요?',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는 걸 기억하겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 44, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWaoL7Qw69ZOQZtfCOhF0uClExvtg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d001cda5-01ef-4ca1-9267-2f9b428f5918-0', usage_metadata={'input_tokens': 44, 'output_tokens': 411, 'total_tokens': 455, 'input_token_details': {'audio': 0, 'cac

In [25]:
for checkpoint in checkpointer.list(config):
  print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b615a-937a-64de-8006-9c0292818969'}}, checkpoint={'v': 4, 'ts': '2025-10-31T04:54:19.804983+00:00', 'id': '1f0b615a-937a-64de-8006-9c0292818969', 'channel_versions': {'__start__': '00000000000000000000000000000006.0.182959251452396', 'query': '00000000000000000000000000000006.0.182959251452396', 'messages': '00000000000000000000000000000008.0.8746570492036349', 'branch:to:add_message': '00000000000000000000000000000007.0.8201523788173334', 'branch:to:llm_response': '00000000000000000000000000000008.0.8746570492036349'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000005.0.15625947711196886'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000006.0.182959251452396'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000007.0.8201523788173334'}}, 'updated_channels': ['messages'], 'channel_values'

In [26]:
print_checkpoint_dump(checkpointer, config)

체크포인트 데이터:
{'channel_values': {'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는 걸 기억하겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 44, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWaoL7Qw69ZOQZtfCOhF0uClExvtg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d001cda5-01ef-4ca1-9267-2f9b428f5918-0', usage_metadata={'input_tokens': 44, 'output_tokens': 411, 'to

In [28]:
# thread_id를 변경해 실행한 경우
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="제가 좋아하는 것은 뭔가요?")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '제가 좋아하는 것은 뭔가요?',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 좋아하는 것은 뭔가요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='제가 당신의 취향은 알 수 없어요. 어떤 영역에서 알고 싶나요? 예: 음식, 영화/음악, 취미 등.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 552, 'prompt_tokens': 37, 'total_tokens': 589, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWbVQSjl0tTV0OBgGfoHK49QaXzRT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c94b71b9-826b-417b-9da7-a107a9ff73af-0', usage_metadata={'input_tokens': 37, 'output_tokens': 552, 'total_tokens': 589, 'input_token_details': {'audio